# Base Lucro Bruto

Esta base prepara uma tabela .parquet (s3a://smartpricinggerdaubucket/SPG_LB/LB_FULL) 

O resultado final contém as colunas que copõe o Lucro Bruto e colunas auxiliares:

GROSS_PROFIT = NET_SALES + COGS_TOTAL + FREIGHT + SHIP_DEL_LOAD_C_C + PORT_EXPENSES + COGS_ADJUSTMENTS

A tabela esta sumarizada no nível: [DATA DA FATURA  + DOCUMENTO DE VENDAS + ITEM DO DOCUMENTO DE VENDAS]

Todo código utiliza PySpark

## Imports & Configs

In [2]:
#imports Libs

import os
import pandas as pd
import boto3
import time
from botocore.client import ClientError

import pyarrow.parquet as pq
import s3fs

import findspark                                              #Import library to Search for Spark Installation  

findspark.init()                                              #Search Spark Installation

import pyspark                                                #Only run after findspark.init()

from pyspark.sql import SparkSession                          #Import of Spark Session
from pyspark import SparkContext as spark                     #Import the Regular Spark Contex 
from pyspark.sql import SQLContext                            #Import the SQL Spark Contex 
from pyspark.sql.window import Window
from pyspark.sql.functions import *
from pyspark.sql import functions as F
from pyspark.sql.types import *
from pyspark.sql.functions import year, month, dayofmonth
spark = SparkSession.builder.getOrCreate()

sc = spark.sparkContext                                       #Initialize Spark

VBox()

Starting Spark application


ID,YARN Application ID,Kind,State,Spark UI,Driver log,Current session?
0,application_1576246565380_0001,pyspark,idle,Link,Link,✔


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

SparkSession available as 'spark'.


FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

-------------

# Creating Variables

## Database

In [ ]:
ATHENA_SPG = os.environ['SPG_DATABASE']

In [ ]:
ATHENA_HANA = os.environ['GERDAU_HANA_DATABASE']

In [ ]:
ATHENA_SALES = os.environ['GERDAU_SALES_DATABASE']

## Buckets

In [ ]:
SPG_MANUAL_INPUT_BUCKET = os.environ['MANUAL_INPUT_BUCKET']

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET = os.environ['INTEGRATION_INPUT_BUCKET']

In [ ]:
SPG_QUERY_BUCKET = os.environ['QUERY_BUCKET']

In [ ]:
GERDAU_BUCKET = os.environ['GERDAU_BUCKET']

In [ ]:
SPG_OUTPUT_BUCKET = os.environ['OUTPUT_BUCKET']

In [ ]:
SPG_INPUT_BUCKET = os.environ['INPUT_BUCKET_FROM_OUTPUT']

## Input Paths

In [ ]:
QUERY_CLIENT = "SELECT DISTINCT cast(O.cod_client as bigint) as cod_client,O.organization_sales_cod,O.registration_date,O.state FROM db_smart_pricing.tb_spg_cl_cliente AS O INNER JOIN (SELECT DISTINCT cod_client,organization_sales_cod,max(registration_date) AS max_date FROM db_smart_pricing.tb_spg_cl_cliente GROUP BY  cod_client, organization_sales_cod) AS S ON S.cod_client=O.cod_client AND S.organization_sales_cod=O.organization_sales_cod AND S.max_date=O.registration_date;"

In [ ]:
SPG_INTEGRATION_INPUT_BUCKET_BW = "SPG_FACTS/SPG_BW/SPG_COPA.parquet"

## Output Paths

In [ ]:
SPG_OUTPUT_BUCKET_GP = "SPG_LB/LB_FULL"

## Boto3 Variables

In [ ]:
#S3 Configuration
S3_ATHENA_INPUT =  's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
S3_ATHENA_OUTPUT = 's3://'+SPG_QUERY_BUCKET+'/'+SPG_QUERY_BUCKET_ATHENA

In [ ]:
region_name = os.environ['AWS_REGION']

In [ ]:
aws_access_key_id = os.environ['AWS_ACCESS_KEY']

In [ ]:
aws_secret_access_key = os.environ['AWS_SECRET_KEY']

-------------

# Creating Defined Functions

In [4]:
# Run Query

def run_query(query, database, s3_output):
    response = client.start_query_execution(
        QueryString=query,
        QueryExecutionContext={
            'Database': database
            },
        ResultConfiguration={
            'OutputLocation': s3_output,
            }
        )
    return response

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [ ]:
def get_aws_path(query,database,s3_output):
    response = run_query(query, database, s3_output)
    file_query = response['QueryExecutionId']
    file_metadata = response['QueryExecutionId'] + '.metadata'
    return file_query

In [ ]:
# Wating for 300 seconds until the end of the upload

def wait_athena_load(Bucket, Key):
    time_to_wait = 300
    time_counter = 0

    while True:
        try:
            s3.meta.client.head_object(Bucket=Bucket,Key=Key)
        except ClientError:
            time.sleep(1)
            time_counter += 1
            if time_counter > time_to_wait:
                break
        else:
            break

-------

# Configuring Boto3

In [ ]:
#Athena Client Configuration

client = boto3.client('athena', 
    aws_access_key_id = aws_access_key_id, 
    aws_secret_access_key = aws_secret_access_key, 
    region_name = region_name )

In [ ]:
#S3 Resource Configuration

s3 = boto3.resource('s3',
    aws_access_key_id = aws_access_key_id,
    aws_secret_access_key = aws_secret_access_key,
    region_name = region_name)

----------

# Importing Tables

In [8]:
#BASE FULL BW COPA

df_bw = spark.read.parquet("s3a://"+SPG_INTEGRATION_INPUT_BUCKET+"/"+SPG_INTEGRATION_INPUT_BUCKET_BW)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [9]:
df_bw.write.partitionBy("GTC100362")
df_bw = df_bw.repartition("GTC100362")
df_bw.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [5]:
# Import CSV from View

athena_response = get_aws_path(QUERY_CLIENT,ATHENA_SPG,S3_ATHENA_OUTPUT)

wait_athena_load(SPG_QUERY_BUCKET, SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [7]:
# Import CSV from View

df_client = spark.read.csv("s3a://"+SPG_QUERY_BUCKET+"/"+SPG_QUERY_BUCKET_ATHENA+"/"+athena_response+".csv", header = 'true')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

--------

## Prepara Tabela

In [12]:
#Transforma colunas em int64

for col_name in ['GTK101129',
                'GTK101085',
                'GTK101087',
                'GTK101096',
                'GTK101231',
                'GTK101126',
                'GTK101238',
                'GTK101124',
                'GTK101099',
                'GTK101236',
                'GTK103147',
                'GTK103148',
                'GTK103149',
                'GTK101118',
                'GTK101115',
                'GTK101112',
                'GTK101119',
                'GTK101120',
                'GTK101121',
                'GTK101125',
                'GTK101086',
                'GTK101111',
                'GTK101113',
                'GTK102652',
                'GTK101093',
                'GTK101123',
                'GTK101094',
                'GTK101095',
                'GTK101127',
                'GTK101233',
                'GTK102651',
                'GTK101088',
                'GTK103150',
                'GTK101091',
                'GTK101226',
                'GTK101103',
                'GTK101098',
                'GTK101221',
                'GTK101222',
                'GTK101229',
                'GTK101097',
                'GTK101100',
                'GTK101092',
                'GTK101116',
                'GTK101117',
                'GTK101114',
                'GTK101223',
                'GTK101090',
                'GTK101101',
                'GTK101122',
                'GTK101224',
                'GTK101102',
                'GTK101225',
                'GTK101105',
                'GTK101227',
                'GTK101104',
                'GTK101106',
                'GTK101228',
                'GTK101107',
                'GTK101108',
                'GTK101109',
                'GTK101110']:
    df_bw = df_bw.withColumn(col_name, col(col_name).cast('float'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Calculo dos Campos

In [13]:
df_bw=df_bw.withColumn("CUSTOOPERACIONAL", col('gtk101088') +
                                                    col('gtk101092') +
                                                    col('gtk101093') +
                                                    col('gtk101094') +
                                                    col('gtk101095') +
                                                    col('gtk101096') +
                                                    col('gtk101229') +
                                                    col('gtk101226') +
                                                    col('gtk101222') +
                                                    col('gtk101228') +
                                                    col('gtk101221') +
                                                    col('gtk101225') +
                                                    col('gtk101224') +
                                                    col('gtk101223') +
                                                    col('gtk101227') +
                                                    col('gtk101231') +
                                                    col('gtk103150') +
                                                    col('gtk101091'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [14]:
df_bw=df_bw.withColumn("Y009_COGS_ADJUSTMENTS", - col('gtk101107') - 
                                                             col('gtk101110') - 
                                                             col('gtk101115') - 
                                                             col('gtk101108') -
                                                             col('gtk101109'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [15]:
df_bw=df_bw.withColumn("Y007_PORT_EXPENSES", col('gtk101087'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [16]:
df_bw=df_bw.withColumn("Y008_SHIP_DEL_LOAD_C_C", - col('gtk101112') - 
                                                          col('gtk101114'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [17]:
df_bw=df_bw.withColumn("Y006_FREIGHT", - col('gtk101090') + 
                                                   col('gtk101086') - 
                                                   col('gtk101127') +
                                                   col('gtk101233') +
                                                   col('gtk102651') - 
                                                   col('gtk102652'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [18]:
df_bw=df_bw.withColumn("GTPM056", - col('gtk103150'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [19]:
df_bw=df_bw.withColumn("YXXX_COGS_TOTAL", (col('gtk101088') + col('gtk101091') + col('gtk101092') + col('gtk101093') + 
                                                    col('gtk101094') + col('gtk101095') + col('gtk101096') + col('gtk101103') + 
                                                    col('gtk101226') + col('gtk101097') + col('gtk101229') + col('gtk101098') +
                                                    col('gtk101221') + col('gtk101099') + col('gtk101222') + col('gtk101100') + 
                                                    col('gtk101223') + col('gtk101101') + col('gtk101224') + col('gtk101102') + 
                                                    col('gtk101225') + col('gtk101104') + col('gtk101105') + col('gtk101227') +
                                                    col('gtk101106') + col('gtk101228') + col('gtk101231')) * (-1))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [20]:
df_bw=df_bw.withColumn("Y002_GROSS_SALES", col('gtk101085') + 
                                                     col('gtk101126') + 
                                                     col('gtk101238') + 
                                                     col('gtk101236') + 
                                                     col('gtk103147') + 
                                                     col('gtk103148') + 
                                                     col('gtk103149'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [21]:
df_bw=df_bw.withColumn("Y003_SALES_TAX_DISCOUNT",      - col('gtk101113') - 
                                                                 col('gtk101116') + 
                                                                 col('gtk101117') + 
                                                                 col('gtk101118') + 
                                                                 col('gtk101119') + 
                                                                 col('gtk101120') + 
                                                                 col('gtk101121') + 
                                                                 col('gtk101122') + 
                                                                 col('gtk101123') - 
                                                                 col('gtk101124'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [22]:
df_bw=df_bw.withColumn("Y004_NET_SALES", col('Y003_SALES_TAX_DISCOUNT') +
                                                   col('Y002_GROSS_SALES'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [23]:
df_bw=df_bw.withColumn("COMISSION_TO_AGENTS",col('gtk101125').cast("float"))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Calculo do Lucro Bruto

In [24]:
df_bw=df_bw.withColumn("GROSS_PROFIT_BW", col('Y004_NET_SALES') +
                                                   col('YXXX_COGS_TOTAL') +
                                                   col('Y006_FREIGHT') +
                                                   col('Y008_SHIP_DEL_LOAD_C_C') +
                                                   col('Y007_PORT_EXPENSES') +
                                                   col('Y009_COGS_ADJUSTMENTS')+
                                                   col('COMISSION_TO_AGENTS'))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Data Prep 

In [25]:
#Seleciona colunas
df_lb = df_bw.select("GTC100511","GTC100020", "GTC101018","GTC100025", "GTC100362", "GTK101129","GTC100209"
                     ,"GTC100255","GTC100504","CUSTOOPERACIONAL", "Y009_COGS_ADJUSTMENTS"
                     ,"Y007_PORT_EXPENSES","Y008_SHIP_DEL_LOAD_C_C","Y006_FREIGHT","GTPM056"
                     ,"YXXX_COGS_TOTAL","Y002_GROSS_SALES","Y003_SALES_TAX_DISCOUNT","Y004_NET_SALES"
                     ,"COMISSION_TO_AGENTS","GROSS_PROFIT_BW")


df_lb = df_lb.withColumnRenamed("GTC100025", 'Data')\
             .withColumnRenamed("GTC100362", 'GPD_cod')\
             .withColumnRenamed("GTK101129", 'Quantity_ton')\
             .withColumnRenamed("GTC100255", 'Sales_Number')\
             .withColumnRenamed("GTC100209", 'cod_client')\
             .withColumnRenamed("GTC100504", 'Sales_Number_Item')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [26]:
df_lb = df_lb.withColumn("Quantity_ton", col('Quantity_ton')/1000)\
             .withColumn("cod_client", col('cod_client').cast("int"))\

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [27]:
df_lb=df_lb.join(df_client.select(df_client.cod_client
                                  ,df_client.organization_sales_cod.alias("GTC100511")
                                  ,df_client.state), on=["GTC100511","cod_client"], how="left")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [28]:
df_lb=df_lb.withColumn("GTC101018", when(trim(df_lb["GTC101018"]).isNull()|df_lb["GTC101018"].like(' '), df_lb.state)\
                                        .otherwise(df_lb["GTC101018"]))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [29]:
df_lb=df_lb.drop("cod_client","state")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [30]:
df_bw.write.partitionBy("GTC101018")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [31]:
df_bw = df_bw.repartition("GTC101018")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [32]:
df_bw.persist(pyspark.StorageLevel.MEMORY_ONLY)

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

DataFrame[0OHREQUID: string, 0DATAPAKID: string, 0RECORD: string, 0FISCVARNT: string, 0FISCPER: string, 0VERSION: string, GTC100202: string, 0VTYPE: string, GTC101700: string, GTC101667: string, GTC101669: string, 0RECORDMODE: string, GTK101128: double, 0G_UABSMG: string, GTC100257: string, GTC100025: string, GTC100505: string, 0CALWEEK: string, GTC100142: string, GTC101698: string, GTC101699: string, GTC100255: string, GTC100504: string, GTC100163: string, GTC100361: string, GTC100878: string, GTC100362: string, GTC100364: string, GTC101655: string, GTC101656: string, GTC101657: string, GTC100450: string, 0CURRENCY: string, GTC101018: string, GTC100190: string, GTC101672: string, GTC100245: string, GTC100194: string, GTC100595: string, GTC100507: string, GTC100506: string, GTC100511: string, GTC100243: string, GTK101129: float, 0G_UVV001: string, GTK101130: double, 0G_UVV002: string, GTK101131: double, 0G_UVV003: string, GTK101132: double, 0G_UVV004: string, GTK101085: float, GTK10108

In [33]:
# Prepara Coluna Data para criação de chave e Groupby
#df_lb = df_lb.withColumn("DataKey",expr("substring(Data, 1, length(Data)-12)"))

# Cria Coluna Mês e Ano

df_lb = df_lb \
            .withColumn('year', year(col('Data'))) \
            .withColumn('month', month(col('Data')))

# Limpa Sales Number
df_lb = df_lb.withColumn("Sales_Number",col("Sales_Number").cast("int").cast("string"))
df_lb = df_lb.withColumn("Sales_Number_Item",col("Sales_Number_Item").cast("int").cast("string"))

# Cria Chaves
df_lb = df_lb.withColumn("KEY_LB", (concat(col("Sales_Number"), lit("_"), col("Sales_Number_Item"))))
df_lb = df_lb.withColumn("KEY_LB_DATA", (concat(col("year"), lit("-"), col("month") ,lit("_"), col("Sales_Number"), lit("_"), col("Sales_Number_Item"))))

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

In [34]:
# Agrupamento por Data + Documento de Vendas + Item do documento de vendas

df_lucro_bruto = df_lb

df_lucro_bruto = df_lucro_bruto.groupby(['KEY_LB_DATA']).agg({'Quantity_ton':'sum','COMISSION_TO_AGENTS': 'sum', 'CUSTOOPERACIONAL': 'sum', 'Y009_COGS_ADJUSTMENTS': 'sum', 'Y007_PORT_EXPENSES': 'sum', 'Y008_SHIP_DEL_LOAD_C_C': 'sum', 'Y006_FREIGHT': 'sum', 'GTPM056': 'sum', 'YXXX_COGS_TOTAL': 'sum', 'Y002_GROSS_SALES': 'sum', 'Y003_SALES_TAX_DISCOUNT': 'sum', 'Y004_NET_SALES': 'sum', 'GROSS_PROFIT_BW': 'sum', 'Data': 'first', 'GPD_cod': 'first','Sales_Number': 'first','Sales_Number_Item': 'first','KEY_LB': 'first', 'GTC101018': 'first', 'GTC100511': 'first', 'GTC100020': 'first'})

# Renomeia devido a infelicidade do PySpark

df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y003_SALES_TAX_DISCOUNT)", 'SALES_TAX_DISCOUNT')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Quantity_ton)", 'QUANTITY_TON')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(CUSTOOPERACIONAL)", 'CUSTO_OPERACIONAL')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(YXXX_COGS_TOTAL)", 'COGS_TOTAL')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y008_SHIP_DEL_LOAD_C_C)", 'SHIP_DEL_LOAD_C_C')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(COMISSION_TO_AGENTS)", 'COMISSION_TO_AGENTS')

df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(GPD_cod)", 'GPD_COD')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(Sales_Number)", 'SALES_NUMBER')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(GTC101018)", 'GTC101018')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(GTC100511)", 'GTC100511')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(GTC100020)", 'GTC100020')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(GROSS_PROFIT_BW)", 'GROSS_PROFIT_BW')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(KEY_LB)", 'KEY_LB')

df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y002_GROSS_SALES)", 'GROSS_SALES')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y004_NET_SALES)", 'NET_SALES')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(GTPM056)", 'GTPM056')

df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(Sales_Number_Item)", 'SALES_NUMBER_ITEM')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y009_COGS_ADJUSTMENTS)", 'COGS_ADJUSTMENTS')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y007_PORT_EXPENSES)", 'PORT_EXPENSES')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("first(Data)", 'DATA')
df_lucro_bruto = df_lucro_bruto.withColumnRenamed("sum(Y006_FREIGHT)", 'FREIGHT')

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…

## Export 

In [35]:
df_lucro_bruto.write.parquet("s3a://"+SPG_OUTPUT_BUCKET+"/"+SPG_OUTPUT_BUCKET_GP, mode = "overwrite")

VBox()

FloatProgress(value=0.0, bar_style='info', description='Progress:', layout=Layout(height='25px', width='50%'),…